In [2]:
import pandas as pd
import csv
import fasttext

In [3]:
from datasets import load_dataset

/Users/srvmdk/Work/SKF/Teaching-Notes/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [19]:
data_dir = '/Users/srvmdk/Work/SKF/Teaching-Notes/data/'

### Load the dataset, you need to login to your huggingface account for this

In [4]:
dataset = load_dataset('datadrivenscience/movie-genre-prediction')

In [5]:
df_train_all = pd.DataFrame(dataset['train'])

In [6]:
df_train_all.head(10)

,id,movie_name,synopsis,genre
0,44978,Super Me,A young scriptwriter starts bringing valuable ...,fantasy
1,50185,Entity Project,A director and her friends renting a haunted h...,horror
2,34131,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...,family
3,78522,Blood Glacier,Scientists working in the Austrian Alps discov...,scifi
4,2206,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...,action
5,46636,Le démon dans l'île,"On an island, things start going crazy as the ...",horror
6,53777,Candid,A video voyeur stalks women in the city with a...,horror
7,24872,Hired,Twin brothers separated at birth and worlds ap...,crime
8,17224,Miruthan,A traffic police officer teams up with his fri...,adventure
9,54900,Until You See Me,A legendary tale unravels.,mystery


In [7]:
df_test_all = pd.DataFrame(dataset['test'])

In [8]:
df_test_all.head(10)

,id,movie_name,synopsis,genre
0,16863,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00...",action
1,48456,Intermedio,A group of four teenage friends become trapped...,action
2,41383,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,84007,Paranoiac,A man long believed dead returns to the family...,action
4,40269,Ordinary Happiness,"After a deadly accident, Paolo comes back on E...",action
5,16524,The Syndicate,"An alcoholic, his wife, a down and out America...",action
6,21245,12 Depths,"""Struck by amnesia and framed for murder, a ma...",action
7,84681,The Feud,A mysterious man returns to his home in Iowa f...,action
8,73352,Cabin Fever: Patient Zero,When a group of friends enjoying a bachelor cr...,action
9,37966,Peregrina,"After a near death experience, a young woman b...",action


### Prepare The Data

In [9]:
df_train_all['text'] = df_train_all['movie_name'] + ' ' + df_train_all['synopsis']
df_test_all['text'] = df_test_all['movie_name'] + ' ' + df_test_all['synopsis']

In [10]:
df_train_all = df_train_all[['text', 'genre']]
df_test_all = df_test_all[['text', 'genre']]

In [11]:
df_train_all.head(10)

,text,genre
0,Super Me A young scriptwriter starts bringing ...,fantasy
1,Entity Project A director and her friends rent...,horror
2,Behavioral Family Therapy for Serious Psychiat...,family
3,Blood Glacier Scientists working in the Austri...,scifi
4,Apat na anino Buy Day - Four Men Widely - Apar...,action
5,"Le démon dans l'île On an island, things start...",horror
6,Candid A video voyeur stalks women in the city...,horror
7,Hired Twin brothers separated at birth and wor...,crime
8,Miruthan A traffic police officer teams up wit...,adventure
9,Until You See Me A legendary tale unravels.,mystery


In [12]:
df_train_all['fasttext_label'] = '__label__' + df_train_all['genre']
df_test_all['fasttext_label'] = '__label__' + df_test_all['genre']

In [13]:
df_train_all.head(10)

,text,genre,fasttext_label
0,Super Me A young scriptwriter starts bringing ...,fantasy,__label__fantasy
1,Entity Project A director and her friends rent...,horror,__label__horror
2,Behavioral Family Therapy for Serious Psychiat...,family,__label__family
3,Blood Glacier Scientists working in the Austri...,scifi,__label__scifi
4,Apat na anino Buy Day - Four Men Widely - Apar...,action,__label__action
5,"Le démon dans l'île On an island, things start...",horror,__label__horror
6,Candid A video voyeur stalks women in the city...,horror,__label__horror
7,Hired Twin brothers separated at birth and wor...,crime,__label__crime
8,Miruthan A traffic police officer teams up wit...,adventure,__label__adventure
9,Until You See Me A legendary tale unravels.,mystery,__label__mystery


In [20]:
df_train_all[['fasttext_label', 'text']].to_csv(data_dir + 'train_full.txt', index = False, sep = ' ', header = None, quoting = csv.QUOTE_NONE, quotechar = "", escapechar = " ")

In [21]:
df_test_all[['fasttext_label', 'text']].to_csv(data_dir + 'test_full.txt', index = False, sep = ' ', header = None, quoting = csv.QUOTE_NONE, quotechar = "", escapechar = " ")

### Train & Test The Fasttext Model

In [22]:
model = fasttext.train_supervised(data_dir + 'train_full.txt', epoch=100)

Read 1M words
Number of words:  115082
Number of labels: 10
Progress: 100.0% words/sec/thread: 3672467 lr:  0.000000 avg.loss:  0.520538 ETA:   0h 0m 0s


In [23]:
model.test(data_dir + 'train_full.txt') # train accuracy

(54000, 0.8582037037037037, 0.8582037037037037)

In [24]:
model.test(data_dir + 'test_full.txt') # test accuracy

(36000, 0.09872222222222223, 0.09872222222222223)